# DATA INGESTION
**Sources* :
- google trends

In [14]:
from pytrends.request import TrendReq

from datetime import date
import pandas as pd
import time

# Google Trends

## KEYWORD LIST

In [15]:
kw_list = ["bitcoin", "btc", "cryptocurrency"] 

## GET THE DATASET

In [16]:
pytrend = TrendReq(hl='en-US', tz=0)

In [62]:
pytrend.build_payload(kw_list, timeframe='2014-09-17 2021-10-20', geo='US')
google_df = pytrend.interest_over_time()
google_df

,bitcoin,btc,cryptocurrency,isPartial
date,,,,
2014-10-01,3,1,0,False
2014-11-01,3,1,0,False
2014-12-01,3,1,0,False
2015-01-01,3,1,0,False
2015-02-01,3,1,0,False
...,...,...,...,...
2021-06-01,31,5,4,False
2021-07-01,22,4,3,False
2021-08-01,20,4,3,False


#### The Limitations
 	--- Last 7 days will have hourly search trends \
 	--- daily data is only provided for query period shorter than 9 months and up to 36 hours before your search \
 	--- Between 9 month and 5 years, and any query longer than 5 years will only return monthly data

In [17]:
today = date.today().strftime("%Y-%m-%d")
today   # ajouter un mois

'2021-10-21'

In [18]:
histo = pd.interval_range(start=pd.Timestamp('2014-08-31'), end=pd.Timestamp(today), freq = '5M', closed = 'both')

In [19]:
for i in histo:
    print(i.left.date(), i.right.date())


2014-08-31 2015-01-31
2015-01-31 2015-06-30
2015-06-30 2015-11-30
2015-11-30 2016-04-30
2016-04-30 2016-09-30
2016-09-30 2017-02-28
2017-02-28 2017-07-31
2017-07-31 2017-12-31
2017-12-31 2018-05-31
2018-05-31 2018-10-31
2018-10-31 2019-03-31
2019-03-31 2019-08-31
2019-08-31 2020-01-31
2020-01-31 2020-06-30
2020-06-30 2020-11-30
2020-11-30 2021-04-30
2021-04-30 2021-09-30


In [20]:
df = []

for i in histo:
    pytrend.build_payload(kw_list, timeframe='{} {}'.format(i.left.date(), i.right.date()), geo='')
    google_df = pytrend.interest_over_time()
    df.append(google_df)
    time.sleep(1)
dfs = pd.concat(df)

In [22]:
dfs

,bitcoin,btc,cryptocurrency,isPartial
date,,,,
2014-08-31,55,16,1,False
2014-09-01,59,15,1,False
2014-09-02,50,13,1,False
2014-09-03,51,18,1,False
2014-09-04,48,18,1,False
...,...,...,...,...
2021-09-26,18,4,2,False
2021-09-27,17,5,2,False
2021-09-28,17,4,2,False


In [ ]:
# ajouter le manque entre le derniere et aujourdui

In [59]:
new_start = (dfs.index.max() + pd.Timedelta(days=1)).strftime("%Y-%m-%d")
new_start


'2021-10-01'

In [63]:
pytrend.build_payload(kw_list, timeframe= '{} {}'.format(new_start, today), geo='')
gap_df = pytrend.interest_over_time()


In [65]:
gap_df

,bitcoin,btc,cryptocurrency,isPartial
date,,,,
2021-10-01,2,0,0,False
2021-10-02,1,0,0,False
2021-10-03,1,0,0,False
2021-10-04,1,0,0,False
2021-10-05,2,0,0,False
2021-10-06,2,0,1,False
2021-10-07,2,0,1,False
2021-10-08,2,0,0,False
2021-10-09,1,0,0,False


In [69]:
gogole_trends_df = pd.concat([dfs, gap_df])

In [70]:
gogole_trends_df

,bitcoin,btc,cryptocurrency,isPartial
date,,,,
2014-08-31,55,16,1,False
2014-09-01,59,15,1,False
2014-09-02,50,13,1,False
2014-09-03,51,18,1,False
2014-09-04,48,18,1,False
...,...,...,...,...
2021-10-15,3,1,0,False
2021-10-16,2,0,0,False
2021-10-17,2,0,0,False


## EXTERNAL STORAGE

In [71]:
gogole_trends_df.to_parquet('/home/giujorge/datalake/lab/Crypto/crypto/data/external/google/google_trends/btc_googletrends_{}.parquet.gzip'.format(date.today().strftime("%Y_%m_%d")), compression='gzip')